### Explaining ReAct Agent in LangChain

- Use for model to decide what to do next.
- Model generates - Thought, Action, Action Input & Observation.
- This loop continues untill it get the final answer.

A ReAct agent is a piece of code that coordinates between reasoning and acting, where:

- The reasoning is carried out by the language model
- The application code performs the acting, at the instruction of the language model.
- This allows problems to be solved by letting a model 'think' through the tasks step-by-step, taking actions and getting action feedback before determining the next steps.

AI Agents operates by decomposing complex goals into manageable subtasks, selecting appropriate tools (such as search engines, database queries, code execution environments, API calls, and other agents) for each subtask, and iteratively executing these tools while analyzing the resulting observations.

ReAct as a methodology for creating AI agents capable of performing quite complex tasks given a set of tools.

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
openai_apikey = os.getenv("OPENAI_API_KEY")
serpapi_apikey = os.getenv("SERP_API")

### Building Product Comparison Assistant

In [6]:
from langchain.tools import Tool
import requests

def search_google_shopping(query):
    api_key = serpapi_apikey
    url = "https://serpapi.com/search"
    params = {
        "engine": "google_shopping",
        "q": query,
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    results = []
    for item in data.get("shopping_results", []):
        results.append(f"{item.get('title')} - ${item.get('price')} at {item.get('source')}")
    return "\n".join(results[:5])  # Limit to top 5 results

serpapi_tool = Tool(
    name="ProductSearchTool",
    func=search_google_shopping,
    description="Search for product prices using Google Shopping via SerpAPI. Input should be a product name like 'iPhone 14'."
)


In [14]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

llm = OpenAI(
temperature=0,
    openai_api_key=openai_apikey)  

agent = initialize_agent(
    tools=[serpapi_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [16]:
user_query = "What is the best price for MacBook Air M2 in India?"
agent.run(user_query)



> Entering new AgentExecutor chain...
 I should use the ProductSearchTool to search for the product price.
Action: ProductSearchTool
Action Input: "MacBook Air M2 India"
Observation: Apple MacBook Air M2 Chip 13-inch - $$229.98 at eBay - epc-smallparts
Apple 13 inch MacBook Air M2 - $$799.00 at Connection
Apple 2023 MacBook Air M2 Chip - $$767.90 at Amazon.com - Seller
Apple MacBook Air 13.6-inch M2 Chip (Refurbished) - $$655.00 at Walmart - Seller
MacBook Air M2 13.6" (2022) - $$849.99 at Mac Star Computers and Camera Store
Thought: I now know the final answer
Final Answer: The best price for MacBook Air M2 in India is $655.00 at Walmart - Seller.

> Finished chain.


'The best price for MacBook Air M2 in India is $655.00 at Walmart - Seller.'